In [18]:
# Import the Inventory_Loss module 
from Inv_Loss_Link import Inventory_Loss

In [19]:
cwd='.' # Defaulted at '.', i.e., the local folder.

# First, instantiate the loss modeul and download sample files if 
# Download=True populated the folder with useful file to run the code; otherwise, those have to be populated by the user
# You may switch Download to False upon downloading files once

Loss=Inventory_Loss(cwd,Download=False)

In [ ]:
# Perform the Z-transformation (https://cheer-hub.github.io/Inventory-Loss-link/Details.html) and save updated tables and Z-transformation dictionary.
Loss.Define_new_table(cwd_look='ProbDS_cwfd_MaxIter3000.pkl')

In [ ]:
# Add a column named "Z" to the inventory, which encodes the configuration of each row for subsequent loss estimation steps.
# It is strongly recommended to read the documentation at "https://cheer-hub.github.io/Inventory-Loss-link/Data_Fusion.html" to understand how the column names 
# (e.g., `blgtype_col`, `roof_col`, etc.) should be defined based on the structure of your inventory file.

#If Mode is set as "Test" it runs the test Z-transfromation data to check the integrity of the code
Loss.Inv_Z_Transformation(
    cwd_inv='NC_Inventory_v1_AI_Imputed',
    blgtype_col=['bldgtype', ['W']],
    roof_col=['roof_shape_1', ['Cross Gable', 'Complex Gable', 'Gable'], ['Hip', 'Cross Hip']],
    story_col=['Story_AI', [1], [2]],
    garage_col=['Garage', [0], [1]],
    res_col=['PRIM_OCC', ['Single Family Dwelling']],
    year_col=['med_yr_blt'],
    Coast_D_col=['Coast_D (miles)', [1]],
    val_struct_col=['val_struct'],
    Mode=''
)

In [ ]:
# Match hazard intensities at building level
# Please refer to the Documentation at "https://cheer-hub.github.io/Inventory-Loss-link/Data_Fusion.html" to get a description of the parameters.
Loss.Hazard_Inv_map(cwd_inv='NC_Inventory_v1_AI_Imputed',cwd_haz='TC_Wise',W_ind=3,I_ind=4,Tr_F=39.3701,Tr_W=2.23694,Average_Mode='Inverse_d4',Mode='')


In [ ]:
# Estimate the loss in M at building level and for regions if specified by the user.
# Please refer to the Documentation at "https://cheer-hub.github.io/Inventory-Loss-link/Data_Fusion.html" to get a description of the parameters.
# Please refer to the Documentation at "https://cheer-hub.github.io/Inventory-Loss-link/Directory.html" that describes the Directory and how different outputs are stored

Loss.Loss_estimate(cwd_haz='TC_Wise',cwd_inv='NC_Inventory_v1_AI_Imputed',interpolation_mode='linear',zone='leaphi_zones_20230920',zone_id='i_lh740')

In [ ]:
# Store and agglomerate loss information into a multi-dimensional tenxor, please refer to the documentation
# to know where it stores and its struct
Loss.Loss_Tensor(cwd_haz='TC_Wise', cwd_inv='NC_Inventory_v1_AI_Imputed',zone='leaphi_zones_20230920',zone_id='i_lh740',interpolation_mode='')